# Tools for Game Theory in GameTheory.jl

**Daisuke Oyama**  
*Faculty of Economics, University of Tokyo*

This notebook demonstrates the functionalities of [GameTheory.jl](https://github.com/QuantEcon/GameTheory.jl).

The first time you run this notebook,
you need to install the Games.jl package by removing the "#" below:

In [1]:
# using Pkg; Pkg.add("GameTheory")

In [2]:
using GameTheory

## Normal Form Games

An $N$-player *normal form game* is a triplet $g = (I, (A_i)_{i \in I}, (u_i)_{i \in I})$ where

* $I = \{1, \ldots, N\}$ is the set of players,
* $A_i = \{1, \ldots, n_i\}$ is the set of actions of player $i \in I$, and
* $u_i \colon A_i \times A_{i+1} \times \cdots \times A_{i+N-1} \to \mathbb{R}$
  is the payoff function of player $i \in I$,

where $i+j$ is understood modulo $N$.

Note that we adopt the convention that the $1$-st argument of the payoff function $u_i$ is
player $i$'s own action
and the $j$-th argument, $j = 2, \ldots, N$, is player ($i+j-1$)'s action (modulo $N$).

In our package,
a normal form game and a player are represented by
the types `NormalFormGame` and `Player`, respectively.

A `Player` carries the player's payoff function and implements in particular
a method that returns the best response action(s) given an action of the opponent player,
or a profile of actions of the opponents if there are more than one.

A `NormalFormGame` is in effect a container of `Player` instances.

### Creating a `NormalFormGame`

There are several ways to create a `NormalFormGame` instance.

The first is to pass an array of payoffs for all the players, i.e.,
an $(N+1)$-dimenstional array of shape $(n_1, \ldots, n_N, N)$
whose $(a_1, \ldots, a_N)$-entry contains an array of the $N$ payoff values
for the action profile $(a_1, \ldots, a_N)$.

As an example, consider the following game ("**Matching Pennies**"):

$
\begin{bmatrix}
1, -1 & -1, 1 \\
-1, 1 & 1, -1
\end{bmatrix}
$

In [3]:
matching_pennies_bimatrix = Array{Float64}(undef, 2, 2, 2)
matching_pennies_bimatrix[1, 1, :] = [1, -1]  # payoff profile for action profile (1, 1)
matching_pennies_bimatrix[1, 2, :] = [-1, 1]
matching_pennies_bimatrix[2, 1, :] = [-1, 1]
matching_pennies_bimatrix[2, 2, :] = [1, -1]
g_MP = NormalFormGame(matching_pennies_bimatrix)

2×2 NormalFormGame{2, Float64}

In [4]:
print(g_MP)

2×2 NormalFormGame{2, Float64}:
 [1.0, -1.0]  [-1.0, 1.0]
 [-1.0, 1.0]  [1.0, -1.0]

In [5]:
g_MP.players[1]  # Player instance for player 1

2×2 Player{2, Float64}:
  1.0  -1.0
 -1.0   1.0

In [6]:
g_MP.players[2]  # Player instance for player 2

2×2 Player{2, Float64}:
 -1.0   1.0
  1.0  -1.0

In [7]:
g_MP.players[1].payoff_array  # Player 1's payoff array

2×2 Matrix{Float64}:
  1.0  -1.0
 -1.0   1.0

In [8]:
g_MP.players[2].payoff_array  # Player 2's payoff array

2×2 Matrix{Float64}:
 -1.0   1.0
  1.0  -1.0

In [9]:
g_MP[1, 1]  # payoff profile for action profile (1, 1)

2-element Vector{Float64}:
  1.0
 -1.0

If a square matrix (2-dimensional array) is given,
then it is considered to be a symmetric two-player game.

Consider the following game (symmetric $2 \times 2$ "**Coordination Game**"):

$
\begin{bmatrix}
4, 4 & 0, 3 \\
3, 0 & 2, 2
\end{bmatrix}
$

In [10]:
coordination_game_matrix = [4 0;
                            3 2]  # square matrix
g_Coo = NormalFormGame(coordination_game_matrix)

2×2 NormalFormGame{2, Int64}

In [11]:
print(g_Coo)

2×2 NormalFormGame{2, Int64}:
 [4, 4]  [0, 3]
 [3, 0]  [2, 2]

In [12]:
g_Coo.players[1].payoff_array  # Player 1's payoff array

2×2 Matrix{Int64}:
 4  0
 3  2

In [13]:
g_Coo.players[2].payoff_array  # Player 2's payoff array

2×2 Matrix{Int64}:
 4  0
 3  2

Another example ("**Rock-Paper-Scissors**"):

$
\begin{bmatrix}
 0,  0 & -1,  1 &  1, -1 \\
 1, -1 &  0,  0 & -1,  1 \\
-1,  1 &  1, -1 &  0,  0
\end{bmatrix}
$

In [14]:
RPS_matrix = [0 -1 1;
              1 0 -1;
              -1 1 0]
g_RPS = NormalFormGame(RPS_matrix)

3×3 NormalFormGame{2, Int64}

In [15]:
print(g_RPS)

3×3 NormalFormGame{2, Int64}:
 [0, 0]   [-1, 1]  [1, -1]
 [1, -1]  [0, 0]   [-1, 1]
 [-1, 1]  [1, -1]  [0, 0]

The second is to specify the sizes of the action sets of the players
to create a `NormalFormGame` instance filled with payoff zeros,
and then set the payoff values to each entry.

Let us construct the following game ("**Prisoners' Dilemma**"):

$
\begin{bmatrix}
1, 1  & -2, 3 \\
3, -2 & 0, 0
\end{bmatrix}
$

In [16]:
g_PD = NormalFormGame((2, 2))  # There are 2 players, each of whom has 2 actions
g_PD[1, 1] = [1, 1]
g_PD[1, 2] = [-2, 3]
g_PD[2, 1] = [3, -2]
g_PD[2, 2] = [0, 0];

In [17]:
print(g_PD)

2×2 NormalFormGame{2, Float64}:
 [1.0, 1.0]   [-2.0, 3.0]
 [3.0, -2.0]  [0.0, 0.0]

Finally, a `NormalFormGame` instance can be constructed by giving an array of `Player` instances,
as explained in the next section.

### Creating a `Player`

A `Player` instance is created by passing an array of dimension $N$
that represents the player's payoff function ("payoff array").

Consider the following game (a variant of "**Battle of the Sexes**"):

$
\begin{bmatrix}
3, 2 & 1, 1 \\
0, 0 & 2, 3
\end{bmatrix}
$

In [18]:
player1 = Player([3 1; 0 2])
player2 = Player([2 0; 1 3]);

Beware that in `payoff_array[h, k]`, `h` refers to the player's own action,
while `k` refers to the opponent player's action.

In [19]:
player1.payoff_array

2×2 Matrix{Int64}:
 3  1
 0  2

In [20]:
player2.payoff_array

2×2 Matrix{Int64}:
 2  0
 1  3

Passing an array of Player instances is the third way to create a `NormalFormGame` instance:

In [21]:
g_BoS = NormalFormGame((player1, player2))

2×2 NormalFormGame{2, Int64}

In [22]:
print(g_BoS)

2×2 NormalFormGame{2, Int64}:
 [3, 2]  [1, 1]
 [0, 0]  [2, 3]

### More than two players

Games with more than two players are also supported.

Let us consider the following version of $N$-player **Cournot Game**.

There are $N$ firms (players) which produce a homogeneous good
with common constant marginal cost $c \geq 0$.
Each firm $i$ simultaneously determines the quantity $q_i \geq 0$ (action) of the good to produce.
The inverse demand function is given by the linear function $P(Q) = a - Q$, $a > 0$,
where $Q = q_1 + \cdots + q_N$ is the aggregate supply.
Then the profit (payoff) for firm $i$ is given by
$$
u_i(q_i, q_{i+1}, \ldots, q_{i+N-1})
= P(Q) q_i - c q_i
= \left(a - c - \sum_{j \neq i} q_j - q_i\right) q_i.
$$

Theoretically, the set of actions, i.e., available quantities, may be
the set of all nonnegative real numbers $\mathbb{R}_+$
(or a bounded interval $[0, \bar{q}]$ with some upper bound $\bar{q}$),
but for computation on a computer we have to discretize the action space
and only allow for finitely many grid points.

The following script creates a `NormalFormGame` instance of the Cournot game as described above,
assuming that the (common) grid of possible quantity values is stored in an array `q_grid`.

In [23]:
function cournot(a::Real, c::Real, ::Val{N}, q_grid::AbstractVector{T}) where {N,T<:Real}
    nums_actions = ntuple(x->length(q_grid), Val(N))
    S = promote_type(typeof(a), typeof(c), T)
    payoff_array= Array{S}(undef, nums_actions)
    for I in CartesianIndices(nums_actions)
        Q = zero(S)
        for i in 1:N
            Q += q_grid[I[i]]
        end
        payoff_array[I] = (a - c - Q) * q_grid[I[1]]
    end
    players = ntuple(x->Player(payoff_array), Val(N))
    return NormalFormGame(players)
end

cournot (generic function with 1 method)

Here's a simple example with three firms,
marginal cost $20$, and inverse demand function $80 - Q$,
where the feasible quantity values are assumed to be $10$ and $15$.

In [24]:
a, c = 80, 20
N = 3
q_grid = [10, 15]  # [1/3 of Monopoly quantity, Nash equilibrium quantity]

g_Cou = cournot(a, c, Val(N), q_grid)

2×2×2 NormalFormGame{3, Int64}

In [25]:
print(g_Cou)

2×2×2 NormalFormGame{3, Int64}:
[:, :, 1] =
 [300, 300, 300]  [250, 375, 250]
 [375, 250, 250]  [300, 300, 200]

[:, :, 2] =
 [250, 250, 375]  [200, 300, 300]
 [300, 200, 300]  [225, 225, 225]

In [26]:
g_Cou.players[1]

2×2×2 Player{3, Int64}:
[:, :, 1] =
 300  250
 375  300

[:, :, 2] =
 250  200
 300  225

In [27]:
g_Cou.nums_actions

(2, 2, 2)

## Nash Equilibrium

A *Nash equilibrium* of a normal form game is a profile of actions
where the action of each player is a best response to the others'.

The `Player` object has methods `best_response` and `best_responses`.

Consider the Matching Pennies game `g_MP` defined above.
For example, player 1's best response to the opponent's action 2 is:

In [28]:
best_response(g_MP.players[1], 2)

2

Player 1's best responses to the opponent's mixed action `[0.5, 0.5]`
(we know they are 1 and 2):

In [29]:
# By default, returns the best response action with the smallest index
best_response(g_MP.players[1], [0.5, 0.5])

1

In [30]:
# With tie_breaking=:random, returns randomly one of the best responses
best_response(g_MP.players[1], [0.5, 0.5], tie_breaking=:random)  # Try several times

2

`best_responses` returns an array of all the best responses:

In [31]:
best_responses(g_MP.players[1], [0.5, 0.5])

2-element Vector{Int64}:
 1
 2

For this game, we know that `([0.5, 0.5], [0.5, 0.5])` is a (unique) Nash equilibrium.

In [32]:
is_nash(g_MP, ([0.5, 0.5], [0.5, 0.5]))

true

In [33]:
is_nash(g_MP, (1, 1))

false

In [34]:
is_nash(g_MP, ([1., 0.], [0.5, 0.5]))

false

### Finding Nash equilibria

There are a few algorithms implemented to compute Nash equilibria:

* Brute force  
  Find all pure-action Nash equilibria of an $N$-player game (if any).
* Sequential best response  
  Find one pure-action Nash equilibrium of an $N$-player game (if any).
* Support enumeration  
  Find all mixed-action Nash equilibria of a two-player nondegenerate game.
* lrsNash  
  Find all mixed-action Nash equilibria (or equilibrium components) of a two-player game.

For more variety of algorithms, one should look at the [`game_theory`](http://quanteconpy.readthedocs.io/en/latest/game_theory.html) module in [QuantEcon.py](https://github.com/QuantEcon/QuantEcon.py) or [Gambit](http://www.gambit-project.org).

#### Brute force

For small games, we can find pure action Nash equilibria by brute force,
by calling the method [`pure_nash`](https://quantecon.github.io/GameTheory.jl/latest/lib/computing_nash_equilibria.html#GameTheory.pure_nash-Tuple{NormalFormGame}).

In [35]:
function print_pure_nash_brute(g::NormalFormGame)
    NEs = pure_nash(g)
    num_NEs = length(NEs)
    if num_NEs == 0
        msg = "no pure Nash equilibrium"
    elseif num_NEs == 1
        msg = "1 pure Nash equilibrium:\n$(NEs[1])"
    else
        msg = "$num_NEs pure Nash equilibria:\n"
        for (i, NE) in enumerate(NEs)
            i < num_NEs ? msg *= "$NE, " : msg *= "$NE"
        end
    end
    println(join(["The game has ", msg]))
end

print_pure_nash_brute (generic function with 1 method)

Matching Pennies:

In [36]:
print_pure_nash_brute(g_MP)

The game has no pure Nash equilibrium


Coordination game:

In [37]:
print_pure_nash_brute(g_Coo)

The game has 2 pure Nash equilibria:
(1, 1), (2, 2)


Rock-Paper-Scissors:

In [38]:
print_pure_nash_brute(g_RPS)

The game has no pure Nash equilibrium


Battle of the Sexes:

In [39]:
print_pure_nash_brute(g_BoS)

The game has 2 pure Nash equilibria:
(1, 1), (2, 2)


Prisoners' Dillema:

In [40]:
print_pure_nash_brute(g_PD)

The game has 1 pure Nash equilibrium:
(2, 2)


Cournot game:

In [41]:
print_pure_nash_brute(g_Cou)

The game has 1 pure Nash equilibrium:
(2, 2, 2)


#### Sequential best response

In some games, such as "supermodular games" and "potential games",
the process of sequential best responses converges to a Nash equilibrium.

Here's a script to find *one* pure Nash equilibrium by sequential best response, if it converges.

In [42]:
function sequential_best_response(g::NormalFormGame{N};
                                  init_actions::Vector{Int}=ones(Int, N),
                                  tie_breaking=:smallest,
                                  verbose=true) where N
    a = copy(init_actions)
    if verbose
        println("init_actions: $a")
    end
    
    new_a = Array{Int}(undef, N)
    max_iter = prod(g.nums_actions)
    
    for t in 1:max_iter
        copyto!(new_a, a)
        for (i, player) in enumerate(g.players)
            if N == 2
                a_except_i = new_a[3-i]
            else
                a_except_i = (new_a[i+1:N]..., new_a[1:i-1]...)
            end
            new_a[i] = best_response(player, a_except_i,
                                     tie_breaking=tie_breaking)
            if verbose
                println("player $i: $new_a")
            end
        end
        if new_a == a
            return a
        else
            copyto!(a, new_a)
        end
    end
    
    println("No pure Nash equilibrium found")
    return a
end

sequential_best_response (generic function with 1 method)

A Cournot game with linear demand is known to be a potential game,
for which sequential best response converges to a Nash equilibrium.

Let us try a bigger instance:

In [43]:
a, c = 80, 20
N = 3
q_grid_size = 13
q_grid = range(0, step=div(a-c, q_grid_size-1), length=q_grid_size)  # [0, 5, 10, ..., 60]
g_Cou = cournot(a, c, Val(N), q_grid)

13×13×13 NormalFormGame{3, Int64}

In [44]:
a_star = sequential_best_response(g_Cou)  # By default, start with (1, 1, 1)
println("Nash equilibrium indices: $a_star")
println("Nash equilibrium quantities: $(q_grid[a_star])")

init_actions: [1, 1, 1]
player 1: [7, 1, 1]
player 2: [7, 4, 1]
player 3: [7, 4, 2]
player 1: [5, 4, 2]
player 2: [5, 4, 2]
player 3: [5, 4, 3]
player 1: [4, 4, 3]
player 2: [4, 4, 3]
player 3: [4, 4, 4]
player 1: [4, 4, 4]
player 2: [4, 4, 4]
player 3: [4, 4, 4]
Nash equilibrium indices: [4, 4, 4]
Nash equilibrium quantities: [15, 15, 15]


In [45]:
# Start with the largest actions (13, 13, 13)
sequential_best_response(g_Cou, init_actions=[13, 13, 13])

init_actions: [13, 13, 13]
player 1: [1, 13, 13]
player 2: [1, 1, 13]
player 3: [1, 1, 7]
player 1: [4, 1, 7]
player 2: [4, 2, 7]
player 3: [4, 2, 5]
player 1: [4, 2, 5]
player 2: [4, 3, 5]
player 3: [4, 3, 4]
player 1: [4, 3, 4]
player 2: [4, 4, 4]
player 3: [4, 4, 4]
player 1: [4, 4, 4]
player 2: [4, 4, 4]
player 3: [4, 4, 4]


3-element Vector{Int64}:
 4
 4
 4

The limit action profile is indeed a Nash equilibrium:

In [46]:
is_nash(g_Cou, tuple(a_star...))

true

In fact, the game has other Nash equilibria
(because of our choice of grid points and parameter values):

In [47]:
print_pure_nash_brute(g_Cou)

The game has 7 pure Nash equilibria:
(5, 4, 3), (4, 5, 3), (5, 3, 4), (4, 4, 4), (3, 5, 4), (4, 3, 5), (3, 4, 5)


Make it bigger:

In [48]:
N = 4
q_grid_size = 61
q_grid = range(0, step=div(a-c, q_grid_size-1), length=q_grid_size)  # [0, 1, 2, ..., 60]
g_Cou = cournot(a, c, Val(N), q_grid)

61×61×61×61 NormalFormGame{4, Int64}

In [49]:
sequential_best_response(g_Cou)

init_actions: [1, 1, 1, 1]
player 1: [31, 1, 1, 1]
player 2: [31, 16, 1, 1]
player 3: [31, 16, 8, 1]
player 4: [31, 16, 8, 5]
player 1: [18, 16, 8, 5]
player 2: [18, 17, 8, 5]
player 3: [18, 17, 12, 5]
player 4: [18, 17, 12, 9]
player 1: [13, 17, 12, 9]
player 2: [13, 15, 12, 9]
player 3: [13, 15, 14, 9]
player 4: [13, 15, 14, 11]
player 1: [12, 15, 14, 11]
player 2: [12, 14, 14, 11]
player 3: [12, 14, 14, 11]
player 4: [12, 14, 14, 12]
player 1: [12, 14, 14, 12]
player 2: [12, 13, 14, 12]
player 3: [12, 13, 14, 12]
player 4: [12, 13, 14, 13]
player 1: [12, 13, 14, 13]
player 2: [12, 13, 14, 13]
player 3: [12, 13, 13, 13]
player 4: [12, 13, 13, 13]
player 1: [13, 13, 13, 13]
player 2: [13, 13, 13, 13]
player 3: [13, 13, 13, 13]
player 4: [13, 13, 13, 13]
player 1: [13, 13, 13, 13]
player 2: [13, 13, 13, 13]
player 3: [13, 13, 13, 13]
player 4: [13, 13, 13, 13]


4-element Vector{Int64}:
 13
 13
 13
 13

In [50]:
sequential_best_response(g_Cou, init_actions=[1, 1, 1, 31])

init_actions: [1, 1, 1, 31]
player 1: [16, 1, 1, 31]
player 2: [16, 8, 1, 31]
player 3: [16, 8, 5, 31]
player 4: [16, 8, 5, 18]
player 1: [17, 8, 5, 18]
player 2: [17, 12, 5, 18]
player 3: [17, 12, 9, 18]
player 4: [17, 12, 9, 13]
player 1: [15, 12, 9, 13]
player 2: [15, 14, 9, 13]
player 3: [15, 14, 11, 13]
player 4: [15, 14, 11, 12]
player 1: [14, 14, 11, 12]
player 2: [14, 14, 11, 12]
player 3: [14, 14, 12, 12]
player 4: [14, 14, 12, 12]
player 1: [13, 14, 12, 12]
player 2: [13, 14, 12, 12]
player 3: [13, 14, 13, 12]
player 4: [13, 14, 13, 12]
player 1: [13, 14, 13, 12]
player 2: [13, 13, 13, 12]
player 3: [13, 13, 13, 12]
player 4: [13, 13, 13, 13]
player 1: [13, 13, 13, 13]
player 2: [13, 13, 13, 13]
player 3: [13, 13, 13, 13]
player 4: [13, 13, 13, 13]


4-element Vector{Int64}:
 13
 13
 13
 13

Sequential best response does not converge in all games:

In [51]:
print(g_MP)  # Matching Pennies

2×2 NormalFormGame{2, Float64}:
 [1.0, -1.0]  [-1.0, 1.0]
 [-1.0, 1.0]  [1.0, -1.0]

In [52]:
sequential_best_response(g_MP)

init_actions: [1, 1]
player 1: [1, 1]
player 2: [1, 2]
player 1: [2, 2]
player 2: [2, 1]
player 1: [1, 1]
player 2: [1, 2]
player 1: [2, 2]
player 2: [2, 1]
No pure Nash equilibrium found


2-element Vector{Int64}:
 2
 1

#### Support enumeration

The routine [`support_enumeration`](https://quantecon.github.io/GameTheory.jl/latest/lib/computing_nash_equilibria.html#GameTheory.support_enumeration-Union{Tuple{NormalFormGame{2,%20T}},%20Tuple{T}}%20where%20T),
which is for two-player games, visits all equal-size support pairs
and checks whether each pair has a Nash equilibrium (in mixed actions) by the indifference condition.
(This should thus be used only for small games.)
For nondegenerate games, this routine returns all the Nash equilibria.

Matching Pennies:

In [53]:
support_enumeration(g_MP)

1-element Vector{Tuple{Vector{Float64}, Vector{Float64}}}:
 ([0.5, 0.5], [0.5, 0.5])

Coordination game:

In [54]:
support_enumeration(g_Coo)

3-element Vector{Tuple{Vector{Float64}, Vector{Float64}}}:
 ([1.0, 0.0], [1.0, 0.0])
 ([0.0, 1.0], [0.0, 1.0])
 ([0.6666666666666666, 0.3333333333333333], [0.6666666666666666, 0.3333333333333333])

Rock-Paper-Scissors:

In [55]:
support_enumeration(g_RPS)

1-element Vector{Tuple{Vector{Float64}, Vector{Float64}}}:
 ([0.3333333333333333, 0.3333333333333333, 0.3333333333333333], [0.3333333333333333, 0.3333333333333333, 0.3333333333333333])

Consider the $6 \times 6$ game by von Stengel (1997), page 12:

In [56]:
player1 = Player(
    [    9504     -660    19976   -20526     1776    -8976;
      -111771    31680  -130944   168124    -8514    52764;
       397584  -113850   451176  -586476    29216  -178761;
       171204   -45936   208626  -263076    14124   -84436;
      1303104  -453420  1227336 -1718376    72336  -461736;
       737154  -227040   774576 -1039236    48081  -300036]
)

player2 = Player(
    [   72336  -461736  1227336 -1718376  1303104  -453420;
        48081  -300036   774576 -1039236   737154  -227040;
        29216  -178761   451176  -586476   397584  -113850;
        14124   -84436   208626  -263076   171204   -45936;
         1776    -8976    19976   -20526     9504     -660;
        -8514    52764  -130944   168124  -111771    31680]
)

g_vonStengel = NormalFormGame(player1, player2);

In [57]:
length(support_enumeration(g_vonStengel))

75

Note that the $n \times n$ game where the payoff matrices are given by the identy matrix
has $2^n−1$ equilibria.
It had been conjectured that this is the maximum number of equilibria of
any nondegenerate $n \times n$ game.
The game above, the number of whose equilibria is $75 > 2^6 - 1 = 63$,
was presented as a counter-example to this conjecture.

Next, let us study the **All-Pay Acution**,
where, unlike standard auctions,
bidders pay their bids regardless of whether or not they win.
Situations modeled as all-pay auctions include
job promotion, R&D, and rent seeking competitions, among others.

Here we consider a version of All-Pay Auction with complete information,
symmetric bidders, discrete bids, bid caps, and "full dissipation"
(where the prize is materialized if and only if
there is only one bidder who makes a highest bid).

Specifically, each of $N$ players simultaneously bids an integer from $\{0, 1, \ldots, c\}$,
where $c$ is the common (integer) bid cap.
If player $i$'s bid is higher than all the other players',
then he receives the prize, whose value is $r$, common to all players,
and pays his bid $b_i$.
Otherwise, he pays $b_i$ and receives nothing (zero value).
In particular, if there are more than one players who make the highest bid,
the prize gets *fully dissipated* and all the players receive nothing.
Thus, player $i$'s payoff function is
$$
u_i(b_i, b_{i+1}, \ldots, b_{i+N-1}) =
\begin{cases}
r - b_i & \text{if $b_i > b_j$ for all $j \neq i$}, \\ - b_i & \text{otherwise}.
\end{cases}
$$

The following is a script to construct a `NormalFormGame` instance
for the All-Pay Auction game:

In [58]:
function all_pay_auction(r::T, c::Integer, ::Val{N};
                         dissipation::Bool=true) where {N,T<:Real}
    nums_actions = ntuple(x->c+1, Val(N))
    S = typeof(zero(T)/one(T))
    payoff_array= Array{S}(undef, nums_actions)
    num_ties = 0
    for bids in CartesianIndices(nums_actions)
        payoff_array[bids] = -(bids[1]-1)
        num_ties = 1
        for j in 2:N
            if bids[j] > bids[1]
                num_ties = 0
                break
            elseif bids[j] == bids[1]
                if dissipation
                    num_ties = 0
                    break
                else
                    num_ties += 1
                end
            end
        end
        if num_ties > 0
            payoff_array[bids] += r / num_ties
        end
    end
    players = ntuple(x->Player(payoff_array), Val(N))
    return NormalFormGame(players)
end

all_pay_auction (generic function with 1 method)

In [59]:
N = 2
c = 5  # odd
r = 8;

In [60]:
g_APA_odd = all_pay_auction(r, c, Val(N))
print(g_APA_odd)

6×6 NormalFormGame{2, Float64}:
 [0.0, 0.0]  [0.0, 7.0]    [0.0, 6.0]    [0.0, 5.0]    [0.0, 4.0]    [0.0, 3.0]
 [7.0, 0.0]  [-1.0, -1.0]  [-1.0, 6.0]   [-1.0, 5.0]   [-1.0, 4.0]   [-1.0, 3.0]
 [6.0, 0.0]  [6.0, -1.0]   [-2.0, -2.0]  [-2.0, 5.0]   [-2.0, 4.0]   [-2.0, 3.0]
 [5.0, 0.0]  [5.0, -1.0]   [5.0, -2.0]   [-3.0, -3.0]  [-3.0, 4.0]   [-3.0, 3.0]
 [4.0, 0.0]  [4.0, -1.0]   [4.0, -2.0]   [4.0, -3.0]   [-4.0, -4.0]  [-4.0, 3.0]
 [3.0, 0.0]  [3.0, -1.0]   [3.0, -2.0]   [3.0, -3.0]   [3.0, -4.0]   [-5.0, -5.0]

Clearly, this game has no pure-action Nash equilibrium.
Indeed:

In [61]:
pure_nash(g_APA_odd)

Tuple{Int64, Int64}[]

As pointed out by Dechenaux et al. (2006),
there are three Nash equilibria when the bid cap `c` is odd
(so that there are an even number of actions for each player):

In [62]:
support_enumeration(g_APA_odd)

3-element Vector{Tuple{Vector{Float64}, Vector{Float64}}}:
 ([0.5, 0.0, 0.25000000000000006, 0.0, 0.24999999999999994, 0.0], [0.0, 0.25, 0.0, 0.25000000000000006, 0.0, 0.5])
 ([0.0, 0.25, 0.0, 0.25000000000000006, 0.0, 0.5], [0.5, 0.0, 0.25000000000000006, 0.0, 0.24999999999999994, 0.0])
 ([0.125, 0.125, 0.12500000000000003, 0.12499999999999997, 0.125, 0.375], [0.125, 0.125, 0.12500000000000003, 0.12499999999999997, 0.125, 0.375])

In addition to a symmetric, totally mixed equilibrium (the third),
there are two asymmetric, "alternating" equilibria (the first and the second).

If `e` is even, there is a unique equilibrium, which is symmetric and totally mixed.
For example:

In [63]:
c = 6  # even
g_APA_even = all_pay_auction(r, c, Val(N))
support_enumeration(g_APA_even)

1-element Vector{Tuple{Vector{Float64}, Vector{Float64}}}:
 ([0.125, 0.125, 0.12500000000000003, 0.12499999999999997, 0.125, 0.12500000000000003, 0.25], [0.125, 0.125, 0.12500000000000003, 0.12499999999999997, 0.125, 0.12500000000000003, 0.25])

Note:
`support_enumeration` is executed with exact arithmetic for input games with `Rational` payoffs.
For example:

In [64]:
g_Coo_rational = NormalFormGame(Rational{Int}, g_Coo)

2×2 NormalFormGame{2, Rational{Int64}}

In [65]:
print(g_Coo_rational)

2×2 NormalFormGame{2, Rational{Int64}}:
 Rational{Int64}[4//1, 4//1]  Rational{Int64}[0//1, 3//1]
 Rational{Int64}[3//1, 0//1]  Rational{Int64}[2//1, 2//1]

In [66]:
support_enumeration(g_Coo_rational)

3-element Vector{Tuple{Vector{Rational{Int64}}, Vector{Rational{Int64}}}}:
 ([1//1, 0//1], [1//1, 0//1])
 ([0//1, 1//1], [0//1, 1//1])
 ([2//3, 1//3], [2//3, 1//3])

In [67]:
N = 2
c = 5  # odd
r = 8//1  # Rational
g_APA_odd_rational = all_pay_auction(r, c, Val(N))

6×6 NormalFormGame{2, Rational{Int64}}

In [68]:
support_enumeration(g_APA_odd_rational)

3-element Vector{Tuple{Vector{Rational{Int64}}, Vector{Rational{Int64}}}}:
 ([1//2, 0//1, 1//4, 0//1, 1//4, 0//1], [0//1, 1//4, 0//1, 1//4, 0//1, 1//2])
 ([0//1, 1//4, 0//1, 1//4, 0//1, 1//2], [1//2, 0//1, 1//4, 0//1, 1//4, 0//1])
 ([1//8, 1//8, 1//8, 1//8, 1//8, 3//8], [1//8, 1//8, 1//8, 1//8, 1//8, 3//8])

#### lrsNash

The routine [`lsrnash`](https://quantecon.github.io/GameTheory.jl/latest/lib/computing_nash_equilibria.html#GameTheory.lrsnash-Tuple{NormalFormGame{2,%20%3C:Union{Int64,%20Rational}}}) calls the Nash equilibrium computation routine in [`lrslib`](http://cgm.cs.mcgill.ca/~avis/C/lrs.html) (through its Julia wrapper [`LRSLib.jl`](https://github.com/JuliaPolyhedra/LRSLib.jl)), which computes in exact arithmetic all extreme mixed-action Nash equilibria of a two-player game with Integer or Rational payoffs, based on the "lexicographic reverse search" vertex enumeration algorithm.

In [69]:
print(g_Coo)

2×2 NormalFormGame{2, Int64}:
 [4, 4]  [0, 3]
 [3, 0]  [2, 2]

In [70]:
lrsnash(g_Coo)

3-element Vector{Tuple{Vector{Rational{BigInt}}, Vector{Rational{BigInt}}}}:
 ([2//3, 1//3], [2//3, 1//3])
 ([0//1, 1//1], [0//1, 1//1])
 ([1//1, 0//1], [1//1, 0//1])

In [71]:
length(lrsnash(g_vonStengel))

75

In [72]:
lrsnash(g_APA_odd_rational)

3-element Vector{Tuple{Vector{Rational{BigInt}}, Vector{Rational{BigInt}}}}:
 ([1//8, 1//8, 1//8, 1//8, 1//8, 3//8], [1//8, 1//8, 1//8, 1//8, 1//8, 3//8])
 ([1//2, 0//1, 1//4, 0//1, 1//4, 0//1], [0//1, 1//4, 0//1, 1//4, 0//1, 1//2])
 ([0//1, 1//4, 0//1, 1//4, 0//1, 1//2], [1//2, 0//1, 1//4, 0//1, 1//4, 0//1])

## Further Reading

* [A Recursive Formulation of Repeated Games](https://nbviewer.jupyter.org/github/QuantEcon/QuantEcon.notebooks/blob/main/recursive_repeated_games.ipynb)

## References

* E. Dechenaux, D. Kovenock, and V. Lugovskyy (2006),
  "Caps on bidding in all-pay auctions:
  Comments on the experiments of A. Rapoport and W. Amaldoss,"
  Journal of Economic Behavior and Organization 61, 276-283.

* B. von Stengel (1997),
  "[New Lower Bounds for the Number of Equilibria in Bimatrix Games](http://www.maths.lse.ac.uk/personal/stengel/TEXTE/264.pdf)."